# `WasteStream` <a class="anchor" id="top"></a>

- **Prepared by:**
    
    - [Yalin Li](zoe.yalin.li@gmail.com)
    - [Joy Cheung](joycheung1994@gmail.com)

- **Covered topics:**

    - [1. WasteStream](#s1)

---
### Note
More detailed documentation coming soon!

---

In [2]:
import qsdsan as qs
print(f'This tutorial is made with qsdsan v{qs.__version__}.')

This tutorial is made with qsdsan v0.3.2.


## 1. `WasteStream` <a class="anchor" id="s1"></a>
A `WasteStream` object can be created by defining flow rate of each `Component` (similar to the creation of a `Stream`) or through built-in influent characterization models (e.g., by specifying total volumetric flowrate, concentrations of total COD, TKN, etc. together with COD fractions).

In [3]:
qs.WasteStream?

In [4]:
# Before using `WasteStream`, we need to tell qsdsan what components we will be working with
# let's load the default components for the demo purpose
cmps = qs.Components.load_default()
qs.set_thermo(cmps)

In [5]:
ins1 = qs.WasteStream('ins1', X_GAO_Gly=.5, H2O=1000, units='kg/hr')
ins2 = qs.WasteStream('ins2', S_NH4=.1, H2O=1000, units='kg/hr')
ins2

WasteStream: ins2
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (g/hr): S_NH4  100
              H2O    1e+06
 WasteStream-specific properties:
  pH         : 7.0
  Alkalinity : 2.5 mg/L
  TN         : 99.7 mg/L
  TKN        : 99.7 mg/L
 Component concentrations (mg/L):
  S_NH4        99.7
  H2O          996859.6


In [6]:
# Default to typical raw wastewater composition based on different models
ws = qs.WasteStream.codstates_inf_model('ws', flow_tot=1000, pH=6.8, COD=500, TP=11)
ws.show(N=20)

C:\Users\joy_c\anaconda3\lib\site-packages\qsdsan\_waste_stream.py:852: UserWarning: ignored concentration specified for H2O:0.0
  warn(f'ignored concentration specified for {bulk_liquid_ID}:{C_h2o}')


WasteStream: ws
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (g/hr): S_F        100
              S_U_Inf    25
              C_B_Subst  46.5
              X_B_Subst  264
              X_U_Inf    65
              X_Ig_ISS   60.8
              S_NH4      25
              S_PO4      8
              S_K        28
              S_Ca       140
              S_Mg       50
              S_CO3      120
              S_N2       18
              S_CAT      3
              S_AN       12
              H2O        9.81e+05
 WasteStream-specific properties:
  pH         : 6.8
  Alkalinity : 10.0 mg/L
  COD        : 500.0 mg/L
  BOD        : 257.9 mg/L
  TC         : 288.7 mg/L
  TOC        : 160.0 mg/L
  TN         : 40.0 mg/L
  TKN        : 40.0 mg/L
  TP         : 11.0 mg/L
  TK         : 28.0 mg/L
 Component concentrations (mg/L):
  S_F          100.0
  S_U_Inf      25.0
  C_B_Subst    46.5
  X_B_Subst    263.5
  X_U_Inf      65.0
  X_Ig_ISS     60.8
  S_NH4        25.0
  S_PO4        8.0
  S_K   

In [7]:
# You can control whether to see Stream information and details
ins1.show(stream_info=False)

WasteStream: ins1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 WasteStream-specific properties:
  pH         : 7.0
  Alkalinity : 2.5 mg/L
  COD        : 495.5 mg/L
  BOD        : 287.4 mg/L
  TC         : 185.8 mg/L
  TOC        : 185.8 mg/L
 Component concentrations (mg/L):
  X_GAO_Gly    495.5
  H2O          991095.9


In [7]:
ins1.show(details=False)

WasteStream: ins1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (g/hr): X_GAO_Gly  500
              H2O        1e+06
 WasteStream-specific properties:
  pH         : 7.0
  Alkalinity : 2.5 mg/L
  ...
 Component concentrations (mg/L):
  X_GAO_Gly    495.5
  H2O          991095.9


In [9]:
ins1.mass
ins1.vol

ws1 = qs.WasteStream()
ws2 = qs.WasteStream()
ins1.split_to(ws1, ws2, .3)
ws1
ws2

ws_mix = qs.WasteStream('ws_mix')
ws_mix.mix_from([ws1, ws2])
ws_mix

WasteStream: ws_mix
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (g/hr): X_GAO_Gly  500
              H2O        1e+06
 WasteStream-specific properties:
  pH         : 7.0
  Alkalinity : 2.5 mg/L
  COD        : 495.5 mg/L
  BOD        : 287.4 mg/L
  TC         : 185.8 mg/L
  TOC        : 185.8 mg/L
 Component concentrations (mg/L):
  X_GAO_Gly    495.5
  H2O          991095.9


In [8]:
# Most composite variables relevant to wastewater treatment are available as properties of the WasteStream object
# ins1.COD
# ins1.TOC
# ins1.TKN
# ins1.TP
# ins1.ThOD

495.54793478617586

In [10]:
# For composite variables related to dissolved or suspended solids, they are 
# available as methods (i.e., functions specifically for the WasteStream class),
# because there are controversies around whether colloidal components should be 
# considered suspened or dissolved solids.
# ws.get_VSS()
ws.get_VSS(include_colloidal=True)

219.39090371144647

All the composite variables above (COD, VSS, etc.) are essentially calculated using the `.composite()` method. You can calculate all kinds of composite variables by specifying different arguments in this method.

In [12]:
?qs.WasteStream.composite

In [14]:
# For example, to calculate the particulate BOD (i.e., xBOD) of the WasteStream object,
# you just need to specify the composite variable as "BOD", and particle size as "x"
ws.composite('BOD', particle_size='x')

152.82952765076288

In [15]:
# biomass COD
ws.composite('COD', specification='X_Bio')

0.0

In [16]:
# nitrogen as nitrate/nitrite 
ws.composite('N', specification='S_NOx')

0.0

In [17]:
# total organic carbon
ws.composite('C', organic=True)

159.9995054905586

In [18]:
# total dissolved solids
ws.composite('solids', particle_size='s')

962.8807802905719

In [19]:
# soluble TKN
ws.composite('N', particle_size='s', specification='TKN')

27.77769192544419

---
### Note
``QSDsan`` can work with three main stream classes: ``Stream``, ``SanStream``, and ``WasteStream``. ``Stream`` is from the package ``Thermosteam`` while ``SanStream`` and ``WasteStream`` are created in ``QSDsan``. The following tutorial is focused on ``WasteStream`` as it is one of the core classes of ``QSDsan``. You can learn more about the different classes in the [documentation](https://qsdsan.readthedocs.io/en/latest/Streams.html).

In the future, it is likely that the `SanStream` class will be merged into `ThermoSteam` so that LCA can be implemented for systems developed using `BioSTEAM`.

---

[Back to top](#top)